Notebook Projet Intelligence Artificielle

Mathieu Boudot - Burak Yalcin - Yann Seget - Gabriel Dufour

L'entreprise de produits pharmaceutiques HumanForYou basée en Inde emploie environ 4000 personnes. Cependant, chaque année elle subit un turn-over d'environ 15% de ses employés nécessitant de retrouver des profils similaires sur le marché de l'emploi.

La direction trouve que ce niveau de turn-over n'est pas bon pour l'entreprise car :

   - Les projets sur lesquels étaient les employés quittant la société prennent du retard ce qui nuit à la réputation de l'entreprise auprès de ses clients et partenaires.
   - Un service de ressources humaines de taille conséquente doit être conservé car il faut avoir les moyens de trouver les nouvelles recrues.
   - Du temps est perdu à l'arrivée des nouveaux employés car ils doivent très souvent être formés et ont besoin de temps pour devenir pleinement opérationnels dans leur nouvel environnement.

Le direction fait donc appel à nous, pour déterminer les facteurs ayant le plus d'influence sur ce taux de turn-over
et lui proposer des modèles afin d'avoir des pistes d'amélioration pour donner à leurs employés l'envie de rester.

### Énoncé du problème
***


Quels sont les facteurs responsables de l'attrition des employés de l'entreprise  ? En effectuant une analyse exploratoire des données (AED), nous pouvons trouver comment chaque caractéristique est responsable de l'attrition des employés de l'entreprise, puis en concevant des modèles ML, trouver la probabilité des caractéristiques pour l'attrition.

### Objectif
***


Nous devons modéliser la probabilité d'attrition à l'aide d'algorithme . Les résultats ainsi obtenus seront utilisés par la direction pour comprendre quels changements elle devrait apporter à son lieu de travail, afin d'inciter la plupart de ses employés à rester.

***



### Sommaire :
***
    - Étape 1: Preprocessing.
    - Étape 2: Pipeline.
    - Étape 3 : Analyse et visualisation statiques des données.
    - Étape 4: Choisir le bon estimateur/algorithme pour notre cas
    - Étape 5 : Pipeline finale

# Data Preprocessing

La première étape consiste à nettoyer et transformer les fichiers que nous avons reçus afin de les rendre exploitables par notre modèle.
Voici un descriptif de différents csv à notre disposition :

- **general_data.csv** : Informations concernant les employés.
- **manager_survey_data.csv** : Evaluation de chaque employé fait par son manager.
- **employee_survey_data.csv** : Enquête qualité de vie au travail.
- **in_time.csv/out_time.csv** : Horaires de travail.

Il faut ensuite les fusionner pour obtenir un DataFrame exploitable par notre algorithme.

## Détection d'anomalies

Avant l'exploitation de nos données, il est nécessaire de passer par une phase d'analyse des fichiers et énumération des anomalies dans leur contenu. Certaines anomalies sont communes à tous les fichiers commes les doublons et les attributs manquants, tandis que d'autres sont propres à chaque fichier.

**Anomalies rencontrées :**
- Doublons.
- Attributs manquants.
- Attributs similaires sur toute une colonne.
- Attributs non numériques.
- Données aberrantes.

# Étape 1: Obtention et importation des Données 
***

En vue de pouvoir exploiter les données, nous les avons traité séparément dans des notebooks et nous les avons enregistrés dans des fichiers CSV pour pouvoir les charger facilement ici :

**Import des librairies**

In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

**Import des données**

In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
warnings.filterwarnings('ignore')

gd = pd.read_csv("./data/general_data.csv")
esd = pd.read_csv("./data/employee_survey_data.csv")
msd = pd.read_csv("./data/manager_survey_data.csv")
in_time = pd.read_csv("./data/in_time.csv")
out_time = pd.read_csv("./data/out_time.csv")

**Transformation des données de temps**

Les données de temps ne sont pas exploitables dans l'état actuel des choses.

Notre objectif est d'obtenir la __moyenne du temps de travail par employé__

Différents points d'attentions : 

* Les dates sont stockées en tant que chaine de caractère et ne sont donc pas exploitables
* Certaines données ne sont pas renseignées (NaN), représentant une absence au travail

Il nous faut donc transformer les dates en objet __datetime__ afin de pouvoir executer des opérations.

Le temps moyen d'un utilisateur au travail lorsqu'il est absent étant de 0, nous pouvons remplacer les NaN par 0.

In [3]:
# Convert in_time and out_time into time type so that we can operate on them
def df_to_time_sheet(df: pd.DataFrame):
    df = df.copy()
    df.rename(columns={df.columns[0]: "EmployeeID"}, inplace=True)
    df.dropna(axis=1, how='all', inplace=True)

    for date in df.columns[1:]:
        df[date] = pd.to_datetime(df[date], errors='ignore', format='%Y-%m-%d %H:%M:%S')

    return df

in_time_sheet = df_to_time_sheet(in_time)
out_time_sheet = df_to_time_sheet(out_time)

In [4]:
work_time_sheet = pd.DataFrame()

# Calculate the defference between the in time and the out time
for date in in_time_sheet.columns[1:]:
    work_time_sheet.insert(len(work_time_sheet.columns), column=date, value=out_time_sheet[date] - in_time_sheet[date])

work_time_sheet.insert(0, column="EmployeeID", value=in_time_sheet[in_time_sheet.columns[0]])

# work_time_sheet.fillna(value=pd.Timedelta(value=0), inplace=True)

Calcule de la moyenne de temps passer au boulot par les employés

In [5]:
work_time_sheet_average = work_time_sheet.drop("EmployeeID", axis=1).mean(axis=1)
work_time_sheet_average = pd.DataFrame(columns=["Average"], data=work_time_sheet_average)
work_time_sheet_average.insert(0, column="EmployeeID", value=work_time_sheet["EmployeeID"])

In [6]:
work_time_sheet_average.head()

,EmployeeID,Average
0,1,0 days 07:22:25.142241379
1,2,0 days 07:43:08.288135593
2,3,0 days 07:00:47.665289256
3,4,0 days 07:11:37.242553191
4,5,0 days 08:00:22.228571428


Calcule du nombre de jours loupé par employés

In [7]:
work_time_sheet_nb_days_off = work_time_sheet.isna().sum(axis=1)
work_time_sheet_nb_days_off = pd.DataFrame(columns=["nb days off"], data=work_time_sheet_nb_days_off)
work_time_sheet_nb_days_off.insert(0, column="EmployeeID", value=work_time_sheet["EmployeeID"])

In [8]:
work_time_sheet_nb_days_off.head()

,EmployeeID,nb days off
0,1,17
1,2,13
2,3,7
3,4,14
4,5,4


Calcule de l'écart type du temps de travail des employés

In [9]:
work_time_sheet_std = pd.DataFrame()

#for name in work_time_sheet.columns:
#    work_time_sheet_std[name] = work_time_sheet[name].values.astype(np.int64, errors='coerce')

work_time_sheet_std = work_time_sheet.astype(np.int64, errors='ignore')
work_time_sheet_std.replace(-9223372036854775808, None, inplace=True)

work_time_sheet_std = work_time_sheet_std.std(axis=1, skipna=True)

work_time_sheet_std = pd.to_timedelta(work_time_sheet_std)
work_time_sheet_std = pd.DataFrame(columns=["Standard deviation"], data=work_time_sheet_std)
work_time_sheet_std.insert(0, column="EmployeeID", value=work_time_sheet["EmployeeID"])

In [10]:
work_time_sheet_std.head()

,EmployeeID,Standard deviation
0,1,26928 days 08:44:25.868357632
1,2,6751 days 14:36:54.475699072
2,3,0 days 00:33:17.437857506
3,4,0 days 00:32:28.933461741
4,5,0 days 00:36:00.584245412


# Fusion de nos différents jeux de données

In [11]:
# Merging main dataset with the work time sheet and surveys

main_dataset = gd.copy()
main_dataset = main_dataset.merge(esd, how='left', on='EmployeeID')
main_dataset = main_dataset.merge(msd, how='left', on='EmployeeID')
main_dataset = main_dataset.merge(work_time_sheet_average,  how='left', on='EmployeeID')
main_dataset = main_dataset.merge(work_time_sheet_nb_days_off,  how='left', on='EmployeeID')
main_dataset = main_dataset.merge(work_time_sheet_std,  how='left', on='EmployeeID')
main_dataset.drop(columns=["EmployeeCount", "EmployeeID", "Over18", "StandardHours"], axis=1, inplace=True)
main_dataset.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Average,nb days off,Standard deviation
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,Married,131160,1.0,11,0,1.0,6,1,0,0,3.0,4.0,2.0,3,3,0 days 07:22:25.142241379,17,26928 days 08:44:25.868357632
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,Single,41890,0.0,23,1,6.0,3,5,1,4,3.0,2.0,4.0,2,4,0 days 07:43:08.288135593,13,6751 days 14:36:54.475699072
2,32,No,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,Married,193280,1.0,15,3,5.0,2,5,0,3,2.0,2.0,1.0,3,3,0 days 07:00:47.665289256,7,0 days 00:33:17.437857506
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,Married,83210,3.0,11,3,13.0,5,8,7,5,4.0,4.0,3.0,2,3,0 days 07:11:37.242553191,14,0 days 00:32:28.933461741
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,Single,23420,4.0,12,2,9.0,2,6,0,4,4.0,1.0,3.0,3,3,0 days 08:00:22.228571428,4,0 days 00:36:00.584245412


In [12]:
main_dataset.to_excel("./buffer/1-main_dataset.xlsx")